In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv("all_cities_edges.csv")

C:\Users\uttam\AppData\Local\Temp\ipykernel_20504\621273052.py:1: DtypeWarning: Columns (8,10,11,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("all_cities_edges.csv")


In [3]:
from shapely import wkt

In [4]:
df1["geometry"] = df1["geometry"].apply(wkt.loads)

In [5]:
df1["latitude"] = df1["geometry"].apply(lambda x: x.coords[0][1])
df1["longitude"] = df1["geometry"].apply(lambda x: x.coords[0][0])

In [6]:
df2 = pd.read_csv("all_cities_nodes.csv")

C:\Users\uttam\AppData\Local\Temp\ipykernel_20504\1735340122.py:1: DtypeWarning: Columns (5,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("all_cities_nodes.csv")


In [7]:
df2.rename(columns={"x": "longitude", "y": "latitude"}, inplace=True)

In [8]:
import geopandas as gpd
from shapely.geometry import Point


In [9]:
# Convert DataFrames to GeoDataFrames
gdf1 = gpd.GeoDataFrame(df1, geometry=gpd.points_from_xy(df1.longitude, df1.latitude))
gdf2 = gpd.GeoDataFrame(df2, geometry=gpd.points_from_xy(df2.longitude, df2.latitude))

# Set a Coordinate Reference System (CRS) for geographical calculations
gdf1.set_crs(epsg=4326, inplace=True)
gdf2.set_crs(epsg=4326, inplace=True)

# Perform a spatial join (find the nearest point from df2 for each point in df1)
merged_gdf = gpd.sjoin_nearest(gdf1, gdf2, how="inner", distance_col="distance")


# Select important features
important_features = [
    "latitude_right","longitude_right","highway_left", "junction_left", "lanes", "maxspeed", "street_count", "oneway","city_left"
]

final_df = merged_gdf[important_features]

C:\Users\uttam\AppData\Roaming\Python\Python313\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [10]:
final_df['lanes'] = pd.to_numeric(final_df['lanes'], errors='coerce')
final_df['maxspeed'] = pd.to_numeric(final_df['maxspeed'], errors='coerce')
final_df['street_count'] = pd.to_numeric(final_df['street_count'], errors='coerce')
final_df.fillna(0, inplace=True)

C:\Users\uttam\AppData\Local\Temp\ipykernel_20504\3620179051.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['lanes'] = pd.to_numeric(final_df['lanes'], errors='coerce')
C:\Users\uttam\AppData\Local\Temp\ipykernel_20504\3620179051.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['maxspeed'] = pd.to_numeric(final_df['maxspeed'], errors='coerce')
C:\Users\uttam\AppData\Local\Temp\ipykernel_20504\3620179051.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [11]:
def categorize_traffic(row):
    # Road Type Weighting
    if row['highway_left'] in ['residential', 'living_street', 'unclassified']:
        road_score = 3  # High congestion
    elif row['highway_left'] in ['primary', 'secondary', 'tertiary']:
        road_score = 2  # Medium congestion
    else:
        road_score = 1  # Low congestion
    
    # Lane Contribution
    if row['lanes'] >= 4:
        lane_score = 1  # Low congestion
    elif row['lanes'] in [2, 3]:
        lane_score = 2  # Medium congestion
    else:
        lane_score = 3  # High congestion

    # Junction Contribution
    if row['junction_left'] == 'roundabout':
        junction_score = 2  # Medium congestion
    elif row['junction_left'] in ['circular', 'turnaround']:
        junction_score = 3  # High congestion
    else:
        junction_score = 1  # Low congestion
    
    # Speed Limit Contribution
    if row['maxspeed'] >= 60:
        speed_score = 1  # Low congestion
    elif 30 <= row['maxspeed'] < 60:
        speed_score = 2  # Medium congestion
    else:
        speed_score = 3  # High congestion

    # Street Count Contribution
    if row['street_count'] >= 5:
        street_score = 3  # High congestion
    elif row['street_count'] in [3, 4]:
        street_score = 2  # Medium congestion
    else:
        street_score = 1  # Low congestion

    # Oneway Impact
    oneway_score = 1 if row['oneway'] else 3  # Oneway roads reduce congestion

    # Overall Traffic Score
    total_score = road_score + lane_score + junction_score + speed_score + street_score + oneway_score

    # Define thresholds
    if total_score <= 7:
        return "Low"
    elif total_score <= 12:
        return "Medium"
    else:
        return "High"

# Assuming your DataFrame is named `df`
final_df['traffic_level_category'] = final_df.apply(categorize_traffic, axis=1)

# Display categorized traffic levels
final_df

C:\Users\uttam\AppData\Local\Temp\ipykernel_20504\561209829.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['traffic_level_category'] = final_df.apply(categorize_traffic, axis=1)


,latitude_right,longitude_right,highway_left,junction_left,lanes,maxspeed,street_count,oneway,city_left,traffic_level_category
0,28.559390,77.171000,primary,0,0.0,0.0,4,True,Delhi,Medium
1,28.559390,77.171000,secondary,0,0.0,0.0,4,True,Delhi,Medium
2,28.580619,77.174953,residential,0,0.0,0.0,4,False,Delhi,High
3,28.580619,77.174953,primary_link,0,0.0,0.0,4,True,Delhi,Medium
4,28.580619,77.174953,trunk,0,0.0,0.0,4,True,Delhi,Medium
...,...,...,...,...,...,...,...,...,...,...
1151561,12.896824,77.547741,residential,0,0.0,0.0,1,False,Bengaluru,High
1151562,12.984290,77.582681,secondary,0,3.0,30.0,3,True,Bengaluru,Medium
1151563,13.070294,77.501688,residential,0,0.0,0.0,3,False,Bengaluru,High
1151564,13.070294,77.501688,residential,0,0.0,0.0,3,False,Bengaluru,High


In [12]:
final_df.drop(columns=["highway_left", "junction_left", "lanes", "maxspeed", "street_count", "oneway","city_left"], inplace=True)

C:\Users\uttam\AppData\Local\Temp\ipykernel_20504\2455119012.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.drop(columns=["highway_left", "junction_left", "lanes", "maxspeed", "street_count", "oneway","city_left"], inplace=True)


In [13]:
delivery = pd.read_csv("indian_delivery_transactions.csv")

In [15]:

# Create GeoDataFrames from the provided datasets
delivery_gdf = gpd.GeoDataFrame(delivery, geometry=gpd.points_from_xy(delivery.delivery_lon, delivery.delivery_lat))
fin_df = gpd.GeoDataFrame(final_df, geometry=gpd.points_from_xy(final_df.longitude_right, final_df.latitude_right))

# Set the Coordinate Reference System (CRS) for geographical calculations
delivery_gdf.set_crs(epsg=4326, inplace=True)
fin_df.set_crs(epsg=4326, inplace=True)



# Perform the first spatial join: finding the nearest point from `wemer` for each point in `fdf`
merged_gdf5 = delivery_gdf.sjoin_nearest(fin_df, how="inner")


# # Optionally, drop the geometry column if it's not needed
# final_gdf = final_gdf.drop(columns=["geometry"])

# # Display the final merged GeoDataFrame
# print(final_gdf.head())



C:\Users\uttam\AppData\Roaming\Python\Python313\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [16]:
merged_gdf5 = merged_gdf5.drop_duplicates(subset="transaction_id", keep="first")
merged_gdf5=merged_gdf5.drop(columns=['geometry', 'index_right'])

In [19]:
weather_merge = pd.read_csv("india_weather_final_20250129_194528.csv")
weather_merge= weather_merge[["latitude","longitude","weather_main","temperature"]]

In [21]:
delivery_gdf = gpd.GeoDataFrame(delivery, geometry=gpd.points_from_xy(delivery.delivery_lon, delivery.delivery_lat))
wemer = gpd.GeoDataFrame(weather_merge, geometry=gpd.points_from_xy(weather_merge.longitude, weather_merge.latitude))

# Set the Coordinate Reference System (CRS) for geographical calculations
delivery_gdf.set_crs(epsg=4326, inplace=True)
wemer.set_crs(epsg=4326, inplace=True)



# Perform the first spatial join: finding the nearest point from `wemer` for each point in `fdf`
merged_gdf1 = delivery_gdf.sjoin_nearest(wemer, how="inner")

C:\Users\uttam\AppData\Roaming\Python\Python313\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [22]:
merged_gdf1=merged_gdf1.drop(columns=['geometry', 'index_right'])

In [23]:
x = merged_gdf1
y = merged_gdf5
y_selected_cols = [col for col in y.columns if col not in x.columns] + ['transaction_id']
z = x.merge(y[y_selected_cols], on='transaction_id', how='left')
z = z.drop(columns=['latitude_right', 'longitude_right','latitude','longitude'])
z.to_csv("start_data.csv", index=False)